The $\textbf{first part}$ is to implement the monomials defined in Proposition 2.22. More precisely, to a Young tableau $U$ (that we consider rectangular for simplicity) we associate the monomial

\begin{equation} \label{monomial}
m_U(x_1,\dots,x_d) := \prod_{k=1}^d \frac{(-1)^{\binom{s_k}{2}} \text{sgn}(\tau_{U;k})}{s_k!} \, S_{\lambda^U(k)}(1^{s_k}) \, x_k^{|\lambda^U(k)|}.
\end{equation}

Below we define the relevant quantities at play:

$\bullet$ $s_k$ is the number of times the integer $k$ appears in the tableau $U$.

$\bullet$ We denote by $(r_i^U(k))_{i=1}^{s_k}$ the sequence of row numbers of boxes of $U$ containing the entry $k$, 
ordered by column-reading $U$. Moreover, let $(r_i^{U,\text{ord}}(k))_{i=1}^{s_k}$ be the ordering of $(r_i^U(k))_{i=1}^{s_k}$ in decreasing order. Then, $\tau_{U;k}$ denotes a permutation that sends $(r_i^U(k))_{i=1}^{s_k}$ to $(r_i^{U,\text{ord}}(k))_{i=1}^{s_k}$, that is, $(r_{\tau_{U;k}(i)}^U(k))_{i=1}^{s_k} = (r_i^{U,\text{ord}}(k))_{i=1}^{s_k}$. 
Finally, we define $\lambda^U(k)$ to be the partition
\begin{align} \label{eq:defpartitionlambda}
\lambda^U(k) := \big( r_i^{U,\text{ord}}(k)-s_k+i-1 \big)_{i=1}^{s_k}.
\end{align} 

$\bullet$ $S_{\lambda^U(k)}(1^{s_k})$ denotes the Schur polynomial associated with the partition $\lambda^U(k)$ and evaluated at 1 in each of its $s_k$ variables:
\begin{align*}
S_{\lambda^U(k)}(1^{s_k}) = \prod_{1\leq i < j \leq s_k} \frac{\lambda^U_i(k)-\lambda^U_j(k)+j-i}{j-i}.
\end{align*} 

The $\textbf{second part}$ implements the combinatorial formula for the fused Specht polynomial:

\begin{align} 
\mathcal F_F = |\text{Stab}{\mathfrak{Q}_\lambda}{F}| \sum_{U \in(\mathfrak{Q}_\lambda.F) \backslash W_F} \text{sign}(\sigma_{F;U}) \; m_U,
\end{align}

where

$\bullet$ $F$ is a $\textit{filling}$: it's is a tableau where the entry $k$ appears freely $s_k$ number of times.  

$\bullet$ In words, the set $(\mathfrak{Q}_\lambda.F) \backslash W_F$ consists of all tableaux obtained from F by permuting different entries within columns, except those where the same entry $k$ appears more than once in any given row.

$\bullet$ Finally, $|\text{Stab}{\mathfrak{Q}_\lambda}{F}| = \prod_{k=1}^d\prod_{i=1}^{n_{\text{col}}} p_{i,k}!$, where $n_{\text{col}}$ is the number of columns of the tableau F, and where $p_{i,k}$ is the number of times that the number $k$ appears in the column $i$.


In [3]:
import numpy as np
from sympy.combinatorics import Permutation
from sympy import expand, symbols, prod, Rational, simplify
from math import comb
from sympy.functions.combinatorial.factorials import factorial
from itertools import permutations, product, combinations

In the cell below, we implement four functions which compute $r_i^U(k))_{i=1}^{s_k}$, $(r_i^{U,\text{ord}}(k))_{i=1}^{s_k}$, $\lambda^U(k)$ and $\tau_{U;k}$.

In [4]:
# Young tableaux are implemented as nested lists. 
# For instance, [[1,2],[3,4]] represents the tableau with first row (1,2) and second row (3,4).
# The number of columns in the tableau is len(tableau[0]).
# The number of rows in the tableau is len(tableau).

def row_sequence(tableau, k):
    result = []
    
    # Iterate over the tableau in column order
    for col in range(len(tableau[0])):
        for row in range(len(tableau)):
            # If the entry matches k, add the row number to the result
            if tableau[row][col] == k:
                result.append(row + 1)
    
    return tuple(result)

def row_sequence_descending(tableau, k):
    
    return tuple(sorted(row_sequence(tableau, k), reverse=True))

def partition_lambda(tableau, k):
    sequence = row_sequence_descending(tableau, k)
    return tuple(sequence[i] - len(sequence) + i for i in range(len(sequence)))

def sign_tau(tableau, k):
    # Get the original and sorted sequences
    orig_seq = row_sequence(tableau, k)
    sorted_seq = row_sequence_descending(tableau, k)

    # Create a mapping from elements to their positions in the original sequence
    position_map = {element: i for i, element in enumerate(orig_seq)}

    # Create the permutation that sorts the original sequence
    tau = [position_map[element] for element in sorted_seq]

    # Create a permutation object from which we get the signature
    tau_obj = Permutation(tau)
    return tau_obj.signature()

We now implement the formula for the Schur polynomial.

In [5]:
# Implementing the Schur polynomial of a given partition and evaluated at 1 in all its variables

def S_lambda(n, partition):
    # If the partition is not in decreasing order, or if not all elements are positive, return 0
    if partition != tuple(sorted(partition, reverse=True)) or any(item < 0 for item in partition):
        return 0

    extended_partition = partition + tuple([0] * (n - len(partition)))
    S_lambda = 1
    for i in range(n):
        for j in range(i+1, n):
            S_lambda *= (extended_partition[i] - extended_partition[j] + j - i) / (j - i)
    return S_lambda

We are now ready to implement the formula for the monomial $m_U$.

In [6]:
def monomial_k(tableau, k):
    
    s_k = len(row_sequence(tableau, k))
    coefficient_k = Rational(sign_tau(tableau, k) * (-1)**comb(s_k, 2) / (factorial(s_k)))

    # Create the symbolic variable y_k
    x_k = symbols(f'x{k}')

    # Compute the monomial
    lambda_k = partition_lambda(tableau, k)
    monomial_k = coefficient_k * Rational(S_lambda(s_k, lambda_k)) * x_k**sum(lambda_k)

    return monomial_k

def monomial(tableau):
    
    entries = set(num for row in tableau for num in row)
    return prod(monomial_k(tableau,k) for k in entries)     

We noe begin the second part. We need to compute the sign of the shortest permutation sending one filling to another.

In [7]:
def sign_sigma_FU(F, U):

    # Flatten F and U column-wise (preserving column structure)
    flat_F = [num for col in zip(*F) for num in col]
    flat_U = [num for col in zip(*U) for num in col]

    # Create a mapping from flat_F to its indices
    index_map = {value: idx for idx, value in enumerate(flat_F)}

    # Build the permutation that maps flat_F to flat_U
    permutation = [index_map[value] for value in flat_U]

    # Compute the signature using SymPy
    perm_obj = Permutation(permutation)

    # Return the correct signature
    return perm_obj.signature()

We now construct the set of tableaux $(\mathfrak{Q}_\lambda.F) \backslash W_F$ over which we sum to get the fused Specht polynomials.

In [8]:
def generate_valid_tableaux(F):

    # Extract columns from F
    columns = list(zip(*F))  # Transpose F to get columns as tuples

    # Generate all permutations for each column
    column_permutations = [list(permutations(col)) for col in columns]

    # Combine permutations of all columns
    all_combinations = product(*column_permutations)

    # Use a set to store unique tableaux
    unique_tableaux = set()
    for combo in all_combinations:
        # Transpose columns back into rows
        U = tuple(zip(*combo))  # Use tuple to make rows hashable
        
        # Check for duplicates in rows
        if all(len(set(row)) == len(row) for row in U):
            unique_tableaux.add(U)  # Add the tableau to the set
    
    # Convert back to list of lists for the final result
    return [list(map(list, tableau)) for tableau in unique_tableaux]

We now compute $|\text{Stab}{\mathfrak{Q}_\lambda}{F}|$.

In [9]:
def count_in_column(tableau, k, l):
    count = 0
    for row in tableau:
        if row[l] == k:  # Check if the number in column l matches k
            count += 1
    return factorial(count)

def stab_k(tableau, k):
    
    n_cols = len(tableau[0])  
    counts = [count_in_column(tableau, k, l) for l in range(n_cols)]
    return prod(counts)

def total_stab(tableau):
    entries = set(num for row in tableau for num in row)
    return prod(stab_k(tableau,k) for k in entries)

We are now ready to compute the fused Specht polynomials!

In [10]:
def fused_specht_polynomial(F):
    result = 0
    valid_tableaux = generate_valid_tableaux(F)
    for U in valid_tableaux:
        result +=  sign_sigma_FU(F, U) * monomial(U)
    return simplify(total_stab(F) * result)

We finally want to make sure that this formula works in the simplest case where $s_k$ = 1 for all $k=1,\dots, d$. In this case, the fused Specht polynomial reduces to an ordinary Specht polynomial. 

In [11]:
def specht_polynomial(tableau):

    # Create symbolic variables for x_1, x_2, ..., x_n
    n = max(max(row) for row in tableau)  # Find the largest number in the tableau
    variables = symbols(f'x1:{n+1}')  
    
    polynomial = 1 

    # Compute the Vandermonde determinant associated with each column
    for col in zip(*tableau):  
        col_diff = prod(variables[j-1] - variables[i-1] for i, j in combinations(col, 2))
        polynomial *= col_diff

    return polynomial

In [12]:
T = [[1,2],[3,4],[5,6],[7,8]]
A = expand(specht_polynomial(T))
B = fused_specht_polynomial(T)

print(A-B)

0
